### Read JSON

In [10]:
import json
from pprint import pprint

data = json.load(open('../SciQdataset-23/test.json'))


In [6]:
print("Total Number of Questions are: %d" % len(data))

Total Number of Questions are: 1000


### Automatic Suject Classification

In [25]:
import pickle

pkl = open('/Users/sherryruan/data/glove/glove.6B/glove.6B.300d.pkl', 'rb')
glove = pickle.load(pkl)
print("loaded glove")

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

import nltk
from nltk import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[\w]+')

from nltk.corpus import stopwords
stopwords = set(nltk.corpus.stopwords.words('english'))

subjects = ["biology", "chemistry", "physics", "geology"]
subjects_vec = [glove[s] for s in subjects]

import numpy as np

question_subject = []

def subject(line):
    tokens = tokenizer.tokenize(line.lower())
    filtered_tokens = [t for t in tokens if t not in stopwords]
    scores = [0 for i in range(len(subjects))]
    for token in filtered_tokens:
        for idx, sub_vec in enumerate(subjects_vec):
            if token in glove:
                score = cosine_similarity(sub_vec, glove[token])
                if score > scores[idx]:
                    scores[idx] = score
    #print(scores)
    return (subjects[np.argmax(scores)])

loaded glove


In [30]:
labelled_data = []

for idx, entry in enumerate(data):
    if idx >= 230: # first 230 questions
        break
    entry["subject"] = subject(entry["question"])
    entry["correct_answer"] = [entry["correct_answer"]]
    entry["difficulyt"] = 0
    labelled_data.append(entry)
    

In [31]:
print(len(labelled_data))

230


In [32]:
with open('../SciQdataset-23/200questions.json', 'w') as outfile:
    json.dump(labelled_data, outfile)